In [55]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

**Решение задачи из пункта а.)**

In [56]:
h_0 = 0.01
l_x = 1
l_y = np.sqrt(2)

n_x = int(np.round(l_x / h_0))
n_y = int(np.round(l_y / h_0))

h_x = l_x / n_x
h_y = l_y / n_y

x = np.linspace(h_x / 2, l_x - h_x / 2, n_x)
y = np.linspace(h_y / 2, l_y - h_y / 2, n_y)

X, Y = np.meshgrid(x, y, indexing='ij')

In [57]:
def f(x, y):
  return np.sin(5 * x) * ((25 + np.pi ** 2 * y ** 2) * np.cos(np.pi * y ** 2 / 2) + np.pi * np.sin(np.pi * y ** 2 / 2))


def μ_L(y):
  return 0


def μ_R(y):
  return np.sin(5) * np.cos(np.pi * y ** 2 / 2)


def μ_B(x):
  return np.sin(5 * x)


def μ_T(x):
  return -np.sin(5 * x)


def u_0(x, y):
  return np.sin(5 * x) * np.cos(np.pi * y ** 2 / 2)

In [58]:
def F(X, Y, h_x, h_y):
    res = f(X, Y)

    # i = 1
    res[0, :] += 2 * μ_L(Y[0, :]) / h_x ** 2

    # i = n_x
    res[-1, :] += 2 * μ_R(Y[-1, :]) / h_x ** 2

    # j = 1
    res[:, 0] += 2 * μ_B(X[:, 0]) / h_y ** 2

    # j = n_y
    res[:, -1] += 2 * μ_T(X[:, -1]) / h_y ** 2

    return res


def L_tilda(u, h_x, h_y):
    res = np.zeros_like(u)

    # i > 1
    res[1:, :] -= u[:-1, :] / h_x ** 2

    # i < n_x
    res[:-1, :] -= u[1:, :] / h_x ** 2

    # j > 1
    res[:, 1:] -= u[:, :-1] / h_y ** 2

    # j < n_y
    res[:, :-1] -= u[:, 1:] / h_y ** 2

    return res


def D(u, h_x, h_y):
    res = np.zeros_like(u)

    # i > 1
    res[1:, :] += 1 / h_x ** 2

    # i = 1
    res[0, :] += 2 / h_x ** 2

    # i < n_x
    res[:-1, :] += 1 / h_x ** 2

    # i = n_x
    res[-1, :] += 2 / h_x ** 2

    # j > 1
    res[:, 1:] += 1 / h_y ** 2

    # j = 1
    res[:, 0] += 2 / h_y ** 2

    # j < n_y
    res[:, :-1] += 1 / h_y ** 2

    # j = n_y
    res[:, -1] += 2 / h_y ** 2

    return res

In [59]:
def Jacobi_method(h_x, h_y, ε, max_iter, errors=None):
    u = np.zeros_like(X)
    D_val = D(u, h_x, h_y)
    r = D_val * u + L_tilda(u, h_x, h_y) - F(X, Y, h_x, h_y)

    for i in range(max_iter):
        u = 1 / D_val * (F(X, Y, h_x, h_y) - L_tilda(u, h_x, h_y))
        r = D_val * u + L_tilda(u, h_x, h_y) - F(X, Y, h_x, h_y)
        if errors is not None:
            errors.append(np.linalg.norm(r))
        if np.linalg.norm(r) < ε:
            break
    return u

In [60]:
ε = 1e-4
max_iter = 100000

u_exact = u_0(X, Y)
u_num = Jacobi_method(h_x, h_y, ε, max_iter)

In [61]:
fig = make_subplots(rows=1, cols=2, subplot_titles=["Numerical solution", "Analytical solution"])

fig.add_trace(go.Heatmap(z=u_num.T, colorscale='Magma', x0=0, dx=l_x, y0=0, dy=l_y, colorbar=dict(title='u')), row=1, col=1)
fig.add_trace(go.Heatmap(z=u_exact.T, colorscale='Magma', x0=0, dx=l_x, y0=0, dy=l_y, colorbar=dict(title='u')), row=1, col=2)

fig.update_layout(
    width=1000,
    height=500,
)

fig.update_xaxes(title_text='x', row=1, col=1)
fig.update_xaxes(title_text='x', row=1, col=2)
fig.update_yaxes(title_text='y', row=1, col=1)
fig.update_yaxes(title_text='y', row=1, col=2)

fig.show()

**Вычисление погрешностей и указание порядка аппроксимации**

In [62]:
h_0 = 0.1

h = []
fault_array = []

for i in range(4):
    n_x = int(np.round(l_x / h_0))
    n_y = int(np.round(l_y / h_0))

    h_x = l_x / n_x
    h_y = l_y / n_y

    x = np.linspace(h_x / 2, l_x - h_x / 2, n_x)
    y = np.linspace(h_y / 2, l_y - h_y / 2, n_y)

    X, Y = np.meshgrid(x, y, indexing='ij')

    u_exact = u_0(X, Y)
    u_num = Jacobi_method(h_x, h_y, ε, max_iter)

    fault_array.append(np.max(np.abs(u_exact - u_num)))

    h.append(h_0)

    h_0 /= 2

In [63]:
tg = abs(np.log(fault_array[0]) - np.log(fault_array[3])) / abs(np.log(h[0]) - np.log(h[3]))
print(tg)

fig = go.Figure(data=[go.Scatter(x=h, y=fault_array, mode='lines+markers', line=dict(color='black'), marker=dict(color='black'))])

fig.update_layout(title='Evaluation of the accuracy of the method',
                  xaxis_title='h',
                  yaxis_title='max_err',
                  xaxis=dict(type='log'),
                  yaxis=dict(type='log'))

fig.show()

1.9831298494751057


**Построение зависимости невязки от номера итерации**

In [64]:
h_0 = 0.01
errors = []
u_num = Jacobi_method(h_x, h_y, ε, max_iter, errors)

In [65]:
fig = go.Figure(data=[go.Scatter(x=np.arange(1, len(errors) + 1), y=errors, mode='lines+markers', line=dict(color='black'), marker=dict(color='black', size=2))])

fig.update_layout(title='Residual dependence on iteration number',
                  xaxis_title='iteration number',
                  yaxis_title='residual',
                  yaxis=dict(type='log'))

fig.show()

**Оценка параметра 𝑞, с которым убывает невязка при итерациях**

In [70]:
log_errors = np.log(errors)
iterations = np.arange(1, len(errors) + 1)
coefficients = np.polyfit(iterations[5000:], log_errors[5000:], 1)
log_q = coefficients[0]
original_value = np.exp(log_q)
print(f'Параметр q: {original_value:.6f}')

Параметр q: 0.999421
